In [6]:
# Model calling and intial setup
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import AzureChatOpenAI , AzureOpenAIEmbeddings
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnableConfig

import warnings
warnings.filterwarnings("ignore") 

load_dotenv(override= True)
# Load env
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_CHAT_DEPLIOYMENT_NAME = os.getenv("AZURE_CHAT_DEPLIOYMENT_NAME")
AZURE_EMBEDDING_DEPLIOYMENT_NAME = os.getenv("AZURE_EMBEDDING_DEPLIOYMENT_NAME")

# get all the langsmith based env 
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
WEATHERSTACK_API_KEY = os.getenv("WEATHERSTACK_API_KEY")

parser = StrOutputParser()

llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite" , api_key= GOOGLE_API_KEY)

llm_openai = AzureChatOpenAI(
    model="gpt-4o-mini",                         
    deployment_name=AZURE_CHAT_DEPLIOYMENT_NAME ,  # deployment name in Azure
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_BASE_URL,
    api_version="2024-02-01",
    temperature=0.75
) 

embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large",
                             deployment=AZURE_EMBEDDING_DEPLIOYMENT_NAME,
                             api_key= AZURE_OPENAI_API_KEY,
                             azure_endpoint= AZURE_BASE_URL,
                             api_version="2024-02-01"
                             )
# result = llm_openai.invoke("What are your creater, also what type of LLM are you").content
# print(result)
# llm_gemini.invoke("who is father of india").content

### what need observablity in AI services
- can track latency drop
- we can log complex llm workflow

### Langsmith
- use for obersvalibilt and eval platform , where team can deugs and moniter app performace

### What langsmith trace
 - i/p and o/p
 - all intermediate steps
 - latency 
 - cost 
 - error 
 - tags 
 - metadata 
 - feedback 

## Core Concept
1) Project 
whole project, that is executed mutiple time 

2) Trace 
- each time the project is execute it is a trace

3) Run
- excustion of each trace have mutiple steps, each of the steps is a single run

In [3]:
prompt = PromptTemplate(template="what is the name of india's first PM?")
chain = prompt | llm_openai | parser
result = chain.invoke(input={})

In [4]:
result

"India's first Prime Minister was Jawaharlal Nehru. He served from August 15, 1947, when India gained independence, until his death on May 27, 1964."

In [13]:
# from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

# This is how one can add project name
os.environ['LANGSMITH_PROJECT']= "Sequential LLM App"
# llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash" , api_key= GOOGLE_API_KEY , temperature=0.9)


parser = StrOutputParser()
chain = prompt1 | llm_openai | parser | prompt2 | llm_openai | parser

config = RunnableConfig(
    run_name="sequential_report_generation_v1",
    tags=["llm_app", "report_generation"]
)
result = chain.invoke({'topic': 'Role of Ai in unemplyoment in India'} , config= config)
print(result)

1. **AI's Dual Impact on Employment**: The report highlights that while AI in India can enhance productivity and create new job opportunities in sectors like IT and agriculture, it also poses significant risks of job displacement, particularly in manufacturing, customer service, and other routine task-oriented jobs.

2. **Current Employment Landscape**: With over 1.4 billion people in India and a workforce of around 500 million, the unemployment rate is approximately 7-8%, with a substantial portion of employment in the informal sector lacking job security and benefits.

3. **Job Displacement Risks**: AI-driven automation is leading to significant job losses, especially for roles involving repetitive tasks, while a mismatch in digital skills among the workforce hampers the transition to new jobs created by AI advancements.

4. **Opportunity for Job Creation**: AI is generating demand for new roles such as data scientists and machine learning engineers, and fostering entrepreneurship th

In [15]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

load_dotenv()  # expects OPENAI_API_KEY in .env

PDF_PATH = "Data\ShantnuKumar.pdf"  # <-- change to your PDF filename

# 1) Load PDF
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()  # one Document per page

# 2) Chunk
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
splits = splitter.split_documents(docs)

# 3) Embed + index
vs = FAISS.from_documents(splits, embeddings)
retriever = vs.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# 4) Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer ONLY from the provided context. If not found, say you don't know."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

config = RunnableConfig(
    run_name="rag_example_1",
    tags=["llm_app", "rag_application"]
)
# 5) Chain
def format_docs(docs): return "\n\n".join(d.page_content for d in docs)

parallel = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
})

chain = parallel | prompt | llm_openai | StrOutputParser()

# 6) Ask questions
print("PDF RAG ready. Ask a question (or Ctrl+C to exit).")
q = input("\nQ: ")
ans = chain.invoke(q.strip() ,  config= config)
print("\nA:", ans)

PDF RAG ready. Ask a question (or Ctrl+C to exit).

A: His AWS skills include leveraging serverless AWS architectures to minimize operational costs while ensuring high scalability and security standards.


#### Imp Points
- By Default only Runnable are tracked by LangSmith, what about the other part like chunkig, loading etc

In [ ]:
import os
from dotenv import load_dotenv

from langsmith import traceable  # <-- key import

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

PDF_PATH = "Data\ShantnuKumar.pdf"  # change to your file

@traceable(name="load_pdf")
def load_pdf(path: str):
    loader = PyPDFLoader(path)
    return loader.load()  # list[Document]

@traceable(name="split_documents" , metadata={"text_splitter" : "RecursiveCharacterTextSplitter"})
def split_documents(docs, chunk_size=1000, chunk_overlap=150):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(docs)

@traceable(name="build_vectorstore" , tags=['vectorDB'] , metadata={"vector_store" : "FAISS" })
def build_vectorstore(splits):
    # FAISS.from_documents internally calls the embedding model:
    vs = FAISS.from_documents(splits, embeddings)
    return vs

# You can also trace a “setup” umbrella span if you want:
@traceable(name="setup_pipeline")
def setup_pipeline(pdf_path: str):
    docs = load_pdf(pdf_path)
    splits = split_documents(docs)
    vs = build_vectorstore(splits)
    return vs

# ---------- pipeline ----------

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer ONLY from the provided context. If not found, say you don't know."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# Build the index under traced setup
vectorstore = setup_pipeline(PDF_PATH)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

parallel = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough(),
})

chain = parallel | prompt | llm_openai | StrOutputParser()

# ---------- run a query (also traced) ----------
print("PDF RAG ready. Ask a question (or Ctrl+C to exit).")
q = input("\nQ: ").strip()

# Give the visible run name + tags/metadata so it’s easy to find:
config = {
    "run_name": "pdf_rag_query"
}

ans = chain.invoke(q, config=config)
print("\nA:", ans)

PDF RAG ready. Ask a question (or Ctrl+C to exit).

A: He managed the full lifecycle for the SpiceReclaim booking forecast model, which included initial development, optimization, and cloud deployment. He improved the accuracy of Payload Prediction models and transitioned them to production environments, ensuring reliable performance using AWS services such as EC2, Lambda, S3, and EventBridge. He also investigated and resolved critical bugs in the Booking Forecast model under pressure, just days before the SpiceReclaim launch.


#### Agent Monitering

In [2]:
# Making our own ai agent
from langchain_community.utilities import GoogleSerperAPIWrapper

search = GoogleSerperAPIWrapper()
search.run("Top news in india today")

"Indian Army, UP govt help late Army officer's daughter reclaim ancestral home. The Indian Army and Uttar Pradesh government coordinated with civil authorities ... Was gang-rape angle considered in RG Kar case, Calcutta HC asks CBI · Nagpur violence: Bulldozers will target more buildings soon, says civic official after ... The Economic Times. Is Archana Puran Singh's son leaving their home? · India Today. Who replaces Pakistan if they boycott T20 World Cup 2026 like Bangladesh? Medha Patkar acquitted in defamation case filed by Delhi L-G · High Court asks Centre to get video of Manipur man's killing removed · Patiala House Court allows ... India Today LIVE TV 24X7 | Headlines | English News LIVE | Latest News · India Today TV Live: Minneapolis ICE Shooting | Shashi Tharoor vs Congress | Bangladesh ... Top News · Dhanush and Mrunal Thakur tie the knot? Truth behind video revealed · Ram Gopal Varma dismisses communal bias claim after AR Rahman's remark · Die My ... Delhi court acquits Me

In [14]:
# Creating a tool out of this 
from langchain_core.tools import Tool , tool

search_tool = Tool(
        name="Intermediate_Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )


In [15]:
search_tool.invoke("Weather condition in delhi right now")

'Hourly Weather · 1 PM 59°. rain drop 0% · 2 PM 62°. rain drop 0% · 3 PM 64°. rain drop 0% · 4 PM 65°. rain drop 0% · 5 PM 63°. rain drop 0% · 6 PM 61°. rain ... Today. 64° / 44°. Mostly Sunny. 1%. 8 mph ... Sunny to partly cloudy. Visibility reduced by smoke. High 64F. Winds NW at 5 to 10 mph. ... Overcast. Hazy. Low 44F. New Delhi Extended Forecast with high and low temperatures ... Morning clouds. Feels Like: 65 °F. Humidity: 42%. Precipitation: Rain: 0 Snow: 0. Current Weather ; RealFeel®. 50° ; RealFeel Shade™. 44° ; Max UV Index. 3.0 (Moderate) ; Wind. WNW 8 mph ; Wind Gusts. 17 mph. Current New Delhi weather condition is Mist with real-time temperature (12°C), humidity 54%, wind 13.3km/h, pressure (1022mb), UV (0), visibility (5km) in ... Today (Sun, January 25): Max 18°C / Min 6°C. PARTLY CLOUDY SKY SUSTAINED SURFACE WINDS SPEED REACHING 15-25 KMPH LIKELY MIST DURING NIGHT. Winds: 9.3 km/h WNW. Today. 63° / 46°. Thunderstorms. 100%. 11 mph ... Showers and thunderstorms. Hazy. H

In [18]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={WEATHERSTACK_API_KEY}&query={city}'

  response = requests.get(url)

  return str(response.content)

In [19]:
from langchain_classic.agents import create_react_agent, AgentExecutor
from langchain_classic import hub

# Fetch a langchain promt from hub
promt = hub.pull("hwchase17/react") # reAct agent

In [20]:
# Ceate a reAct agent
agent = create_react_agent(
    llm= llm_openai,
    tools= [search_tool , get_weather_data],
    prompt= promt
)

In [23]:
# Creating an agent executer
agent_executer = AgentExecutor(
    agent= agent , 
    tools=[search_tool , get_weather_data],
    verbose= True
)

In [ ]:
responce = agent_executer.invoke({"input":"What is the Home city of Speed(streamer) and what the temp of that city"})
print(responce)



> Entering new AgentExecutor chain...
I need to find out the home city of Speed, the streamer, and then check the current temperature there. First, I will search for information about Speed's home city.

Action: Intermediate_Answer  
Action Input: "What is the home city of Speed the streamer?"  Watkins at Chinatown, Singapore, in 2024 ; Born. Darren Jason Watkins Jr. (2005-01-21) January 21, 2005 (age 21). Cincinnati, Ohio, U.S. ; Other names. Speed ... Take a virtual tour of IShowSpeed's lavish $10 million Florida house. This celebrity home showcases luxury and opulence, with stunning features and amenities. 🌴 This impressive residence is available for $15,000 per month and boasts an expansive layout of 4,700 square feet. This is the home of $2 million YouTuber iow speed in Davy Florida and you'll never believe what the $3 million room is hiding. I was today years old when I learned IShowSpeed is from Cincinnati ... I'm not into streaming bros at all, but this crazy mofo shows up in

: 

#### LangSmith in Langgraph

In [7]:
import operator
from typing import TypedDict, Annotated, List

from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langsmith import traceable
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

load_dotenv()

class EvaluationSchema(BaseModel):
    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="Score out of 10", ge=0, le=10)

structured_model = llm_gemini.with_structured_output(EvaluationSchema)

essay2 = """India and AI Time

Now world change very fast because new tech call Artificial Intel… something (AI). India also want become big in this AI thing. If work hard, India can go top. But if no careful, India go back.

India have many good. We have smart student, many engine-ear, and good IT peoples. Big company like TCS, Infosys, Wipro already use AI. Government also do program “AI for All”. It want AI in farm, doctor place, school and transport.

In farm, AI help farmer know when to put seed, when rain come, how stop bug. In health, AI help doctor see sick early. In school, AI help student learn good. Government office use AI to find bad people and work fast.

But problem come also. First is many villager no have phone or internet. So AI not help them. Second, many people lose job because AI and machine do work. Poor people get more bad.

One more big problem is privacy. AI need big big data. Who take care? India still make data rule. If no strong rule, AI do bad.

India must all people together – govern, school, company and normal people. We teach AI and make sure AI not bad. Also talk to other country and learn from them.

If India use AI good way, we become strong, help poor and make better life. But if only rich use AI, and poor no get, then big bad thing happen.

So, in short, AI time in India have many hope and many danger. We must go right road. AI must help all people, not only some. Then India grow big and world say "good job India".
"""

class UPSCState(TypedDict, total=False):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[List[int], operator.add]  # merges parallel lists
    avg_score: float

@traceable(name="evaluate_language_fn", tags=["dimension:language"], metadata={"dimension": "language"})
def evaluate_language(state: UPSCState):
    prompt = (
        "Evaluate the language quality of the following essay and provide feedback "
        "and assign a score out of 10.\n\n" + state["essay"]
    )
    out = structured_model.invoke(prompt)
    return {"language_feedback": out.feedback, "individual_scores": [out.score]}

@traceable(name="evaluate_analysis_fn", tags=["dimension:analysis"], metadata={"dimension": "analysis"})
def evaluate_analysis(state: UPSCState):
    prompt = (
        "Evaluate the depth of analysis of the following essay and provide feedback "
        "and assign a score out of 10.\n\n" + state["essay"]
    )
    out = structured_model.invoke(prompt)
    return {"analysis_feedback": out.feedback, "individual_scores": [out.score]}

@traceable(name="evaluate_thought_fn", tags=["dimension:clarity"], metadata={"dimension": "clarity_of_thought"})
def evaluate_thought(state: UPSCState):
    prompt = (
        "Evaluate the clarity of thought of the following essay and provide feedback "
        "and assign a score out of 10.\n\n" + state["essay"]
    )
    out = structured_model.invoke(prompt)
    return {"clarity_feedback": out.feedback, "individual_scores": [out.score]}


@traceable(name="final_evaluation_fn", tags=["aggregate"])
def final_evaluation(state: UPSCState):
    prompt = (
        "Based on the following feedback, create a summarized overall feedback.\n\n"
        f"Language feedback: {state.get('language_feedback','')}\n"
        f"Depth of analysis feedback: {state.get('analysis_feedback','')}\n"
        f"Clarity of thought feedback: {state.get('clarity_feedback','')}\n"
    )
    overall = llm_gemini.invoke(prompt).content
    scores = state.get("individual_scores", []) or []
    avg = (sum(scores) / len(scores)) if scores else 0.0
    return {"overall_feedback": overall, "avg_score": avg}

# ---------- Build graph ----------
graph = StateGraph(UPSCState)

graph.add_node("evaluate_language", evaluate_language)
graph.add_node("evaluate_analysis", evaluate_analysis)
graph.add_node("evaluate_thought", evaluate_thought)
graph.add_node("final_evaluation", final_evaluation)

# Fan-out → join
graph.add_edge(START, "evaluate_language")
graph.add_edge(START, "evaluate_analysis")
graph.add_edge(START, "evaluate_thought")
graph.add_edge("evaluate_language", "final_evaluation")
graph.add_edge("evaluate_analysis", "final_evaluation")
graph.add_edge("evaluate_thought", "final_evaluation")
graph.add_edge("final_evaluation", END)

workflow = graph.compile()


In [8]:
result = workflow.invoke(
    {"essay": essay2},
    config={
        "run_name": "evaluate_upsc_essay",  # becomes root run name
        "tags": ["essay", "langgraph", "evaluation"],
        "metadata": {
            "essay_length": len(essay2),
            "model": "gpt-4o-mini",
            "dimensions": ["language", "analysis", "clarity"],
        },
    },
)

print("\n=== Evaluation Results ===")
print("Language feedback:\n", result.get("language_feedback", ""), "\n")
print("Analysis feedback:\n", result.get("analysis_feedback", ""), "\n")
print("Clarity feedback:\n", result.get("clarity_feedback", ""), "\n")
print("Overall feedback:\n", result.get("overall_feedback", ""), "\n")
print("Individual scores:", result.get("individual_scores", []))
print("Average score:", result.get("avg_score", 0.0))


=== Evaluation Results ===
Language feedback:
 The essay demonstrates a basic understanding of the topic but suffers from significant language quality issues. The vocabulary is overly simplistic, and sentence structures are often fragmented or grammatically incorrect (e.g., "Now world change very fast," "India have many good," "many villager no have phone"). The use of colloquialisms and informal language is prevalent, detracting from the overall tone and clarity. While the core ideas about India's potential and challenges with AI are present, they are not articulated effectively due to these language limitations. For instance, concepts like "AI for All" and the specific applications in farming and healthcare are mentioned but lack detailed explanation or sophisticated phrasing. The essay would greatly benefit from improved grammar, a wider vocabulary, and more complex sentence construction to convey its message more persuasively and professionally. 

Analysis feedback:
 The essay tou

##### Features in LangSmith
- moniter and alerts
- can moniter across traces:
- latency, token_usgaes , cost , error_rate , success_rate
- Evalution :
- test gold-standard dataset, custome metrices like faithfulness, truthfullness

- promt experimetation :
- test diff experiments A/B testing across promts on same dataset

- Dataset creation and annotation 
- has tool to build dataset, eval and fine-tune
- support manual eval

- user feedback integration 
- thumbs up, thumb down rating and structure feedback
- can get bulk analysis

- collaboration 
- has sharing, and create shared experiment dashboards
